In [ ]:
import pydriver
import numpy as np 
import sklearn, sklearn.cluster, matplotlib.pyplot as plt
import copy, datetime 
import glob, os
import pandas as pd
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler


In [2]:
path = "/mnt/storage/home/ja17618/scratch/DATA_DIR/training/"
label_path ='/mnt/storage/home/ja17618/scratch/models/research/deeplab/datasets/labels.csv'
f_lidar = glob.glob(os.path.join(path, 'velodyne', '*.bin'))
f_rgb = glob.glob(os.path.join(path, 'image_2', '*.png'))
f_labels = glob.glob(os.path.join(path, 'label_2', '*.txt'))
data_tag = [name.split('/')[-1].split('.')[-2] for name in f_lidar]
files=[tag+".bin" for tag in data_tag]

In [ ]:
# point cloud coloring (False: reflectance, True: camera)
USE_IMAGE_COLOR = False
reader = pydriver.datasets.kitti.KITTIObjectsReader(path)
reconstructor = pydriver.preprocessing.LidarReconstructor(
    useImageColor=USE_IMAGE_COLOR,
    removeInvisible=True,
    )
# mandatory ground truth categories
DETECTION_CATEGORIES = ['car','van', 'truck', 'pedestrian', 'person_sitting', 'cyclist','tram']
# optional ground truth categories
DETECTION_CATEGORIES_OPT = ['misc', 'dontcare']

MIN_OVERLAP = 0.7               # minimal overlap between 2D boxes
EVALUATION_MODE = 'moderate'    # mode (easy, moderate, hard)
VISUALIZE3D = False  

In [ ]:
SHOT_RADIUS = 2.0
preprocessor = pydriver.preprocessing.Preprocessor(reconstructor)
keypointExtractor = pydriver.keypoints.ISSExtractor(salientRadius=0.25, nonMaxRadius=0.25)
featureExtractor = pydriver.features.SHOTColorExtractor(shotRadius=SHOT_RADIUS, fixedY=-1.0)
featureTypes = [('myfeature', featureExtractor.dims),]


In [ ]:
# function the vocabularies will use to create category storages
def storageGenerator(dims, category):
    sto = pydriver.detectors.vocabularies.Storage(dims, category,
        preprocessors=[],
        regressor=sklearn.neighbors.KNeighborsRegressor(n_neighbors=1),
        )
    return sto
# function the detector will use to create vocabularies
def vocabularyGenerator(dimensions, featureName):
    voc = pydriver.detectors.vocabularies.Vocabulary(
        dimensions,
        preprocessors=[
            sklearn.cluster.MiniBatchKMeans(n_clusters=100, batch_size=10, max_iter=100),
            ],
        classifier=sklearn.ensemble.AdaBoostClassifier(n_estimators=75),
        storageGenerator=storageGenerator,
        balanceNegatives=True,
        )
    return voc

In [ ]:
# initialize detector that will perform learning and recognition
detector = pydriver.detectors.Detector(featureTypes, vocabularyGenerator=vocabularyGenerator)


In [ ]:
import pandas as pd
# context_df = pd.read_÷csv("labels.csv",dtype={'frame': np.int32, 'context': np.str},index_col=0)
# context_df['frame']=context_df['frame'].apply(lambda x: '{0:0>6}'.format(x))

context_df = pd.read_csv(label_path,index_col=False,header=0)

def get_context(frame):
    c_df = context_df[context_df['frame']==frame]
    return str(c_df['context'].values[0])



In [ ]:
# for image,pcl,label in zip(f_rgb,f_lidar,f_labels):
#     print("File Number %s" %(image.split('/')[-1].split('.')[-2]))
#     raw_lidar = np.fromfile(pcl,dtype=np.float32).reshape((-1,4))
#     scene = preprocessor.process(raw_lidar)
#     input()
np.set_printoptions(threshold=np.inf)
timeStart = datetime.datetime.now()
for frame in reader.getFramesInfo(1,10):
    print('Training on frame %d...' % frame['frameId'])
    scene = preprocessor.process(frame)
    keypointCloud = keypointExtractor.getKeypointCloud(scene)
    fkeypoints, features = featureExtractor.getFeatures(scene, keypointCloud)
#     print(fkeypoints)
    groundTruth, groundTruthOpt = pydriver.datasets.kitti.getKITTIGroundTruth(
        frame['labels'],
        DETECTION_CATEGORIES,
        DETECTION_CATEGORIES_OPT,
        mode='moderate',    # use moderate mode for training
        )
    category = get_context(frame['frameId'])
    boxes3D_exclude = []
    for label in groundTruth + groundTruthOpt:
        box3D = pydriver.geometry.transform3DBox(label['box3D'], scene['transformation'])
        # avoid training with non-negative examples
        # the box is made bigger so SHOT features used as negatives
        # don't capture parts of the object
        box3D_exclude = copy.deepcopy(box3D)
        box3D_exclude['dimensions']['height'] += 2*SHOT_RADIUS
        box3D_exclude['dimensions']['width'] += 2*SHOT_RADIUS
        box3D_exclude['dimensions']['length'] += 2*SHOT_RADIUS
        boxes3D_exclude.append(box3D_exclude)
        if category == 'urban':
            if label in groundTruth:
                # get keypoints which lie inside the labeled object box
                boxKeypointCloud = keypointCloud.extractOrientedBoxes([box3D])
                # extract features at these keypoints (and get new keypoints
                # which depend on the feature extractor)
                fkeypoints, features = featureExtractor.getFeatures(scene, boxKeypointCloud)
                # learn new features and relations between features and objects
                detector.addWords(category, 'myfeature', features, fkeypoints, box3D)

        
        # get keypoints which lie outside of labeled object boxes
    negativeKeypointCloud = keypointCloud.extractOrientedBoxes(boxes3D_exclude, invert=True)
    # extract features at these keypoints
    fkeypoints, features = featureExtractor.getFeatures(scene, negativeKeypointCloud)
    # learn features associated with absence of objects
    detector.addWords('negative', 'myfeature', features)
            
timeTraining = datetime.datetime.now() - timeStart 
# perform learning on stored data
print('Learning...')
timeStart = datetime.datetime.now()
detector.learn(nStorageMaxRandomSamples=25000)
timeLearning = datetime.datetime.now() - timeStart


In [ ]:
evaluator = pydriver.evaluation.Evaluator(minOverlap=MIN_OVERLAP, nPoints=100)
# perform testing with frames which were not used for training
firstFrame = 10
lastFrame = 10 + 10
timeStart = datetime.datetime.now()
for frame in reader.getFramesInfo(firstFrame, lastFrame):
    print('Testing on frame %d...' % frame['frameId'])
    # see the training part above
    scene = preprocessor.process(frame)
    keypointCloud = keypointExtractor.getKeypointCloud(scene)
    groundTruth, groundTruthOpt = pydriver.datasets.kitti.getKITTIGroundTruth(
        frame['labels'],
        DETECTION_CATEGORIES,
        DETECTION_CATEGORIES_OPT,
        mode=EVALUATION_MODE,
        )

    # extract keypoints and features for the whole scene
    fkeypoints, features = featureExtractor.getFeatures(scene, keypointCloud)
    # perform recognition on extracted features
    detections = detector.recognize({'myfeature': (fkeypoints,features)})

    # convert 3D detections (NumPy array) to labels (list of
    # dictionaries) that include 2D box projections used for evaluation
    # and revert the transformation of the scene, so they have the same
    # coordinate system as the original KITTI labels
    detections_labels = pydriver.datasets.detections2labels(
        detections,
        scene['transformation'].I,    # inverse matrix
        frame['calibration']['projection_left'],
        scene['img_left'].shape,
        )
    # exclude detections which are always considered optional by
    # KITTI (i.e. in 'hard' mode) and will not positively contribute
    # to performance
    detections_labels = [l for l in detections_labels if \
                          l['info']['truncated'] <= 0.5 and \
                          l['box2D']['bottom']-l['box2D']['top'] >= 25.0
                        ]

    # add frame recognition results to evaluator
    evaluator.addFrame(groundTruth, groundTruthOpt, detections_labels)
    if VISUALIZE3D:
        # perform visualization in the transformed cloud
        # convert ground truth labels to ground truth detections
        gtd = pydriver.datasets.labels2detections(groundTruth, scene['transformation'])
        gtdOpt = pydriver.datasets.labels2detections(groundTruthOpt, scene['transformation'])
        scene['cloud'].visualizeDetections(detections, gtd, gtdOpt)
timeEvaluation = datetime.datetime.now() - timeStart

# show evaluation results
print("Training time: %s" % timeTraining)
print("Learning time: %s" % timeLearning)
print("Evaluation time: %s" % timeEvaluation)
print("Average precision: %.2f" % evaluator.aprecision)
print("Average orientation similarity: %.2f" % evaluator.aos)
values = evaluator.getValues()
plt.figure()
plt.plot(values['recall'], values['precision'],
    label='Precision (AP %0.2f)' % evaluator.aprecision)
plt.plot(values['recall'], values['OS'],
    label='Orientation similarity (AOS %0.2f)' % evaluator.aos)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision / OS')
plt.legend(loc="upper right")
plt.show()

In [39]:
context_df = pd.read_csv(label_path,index_col=False,header=0)
context_df['frame']=context_df['frame'].apply(lambda x: '{0:0>6}'.format(x))
def get_context(id):
    f = context_df[context_df['frame']==id]
    f = f['context'].values[0]
    return f
x = context_df.loc[:,'frame'].values
y = context_df.loc[:,['context']].values

X = np.array([np.fromfile(pcl,dtype=np.float32).reshape() for pcl in f_lidar[:10]])
data_tag = [name.split('/')[-1].split('.')[-2] for name in f_lidar]
Y = [get_context(id) for id in data_tag]
clf = neighbors.KNeighborsRegressor()
clf.fit(X,Y)

# for pcl in f_lidar[:20]:
#     id = pcl.split('/')[-1].split('.')[-2]
#     x = np.fromfile(pcl,dtype=np.float32).reshape((-1,4))
#     y = get_context(id)
#     clf.fit(x,y)


cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
            edgecolor='k', s=20)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("3-Class classification (k = %i, weights = '%s')"
          % (n_neighbors, weights))

ValueError: Found input variables with inconsistent numbers of samples: [3, 3712]